In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ------- Import Libs ---------
from datetime import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import math

import warnings
import re

import plotly.express as px

alt.renderers.enable("altair_viewer")

# Disbale the max row limit for altair datasets.
alt.data_transformers.disable_max_rows()

In [ ]:
# Set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

print(CWD)

In [ ]:
# -------- Load in the 15 min and hourly data ----------
data_hourly = pd.read_csv("data/island_A/exported/hourly_data_clean.csv")
data_15min = pd.read_csv("data/island_A/exported/data_15min_clean.csv")

fmintohour_MWh = pd.read_csv("data/island_A/exported/15mintohour_MWh.csv")
fmintohour_MW = pd.read_csv("data/island_A/exported/15mintohour_MW.csv")

In [ ]:
# information about the data
def info(data):
    print(data.shape)
    print(data.dtypes)
    print(data.head())


info(data_hourly)
info(data_15min)

In [ ]:
# ------ Selecting a single data to chart -------
solo_date = pd.to_datetime("2018-11-05")

solo_hourly_df = data_hourly[data_hourly["Datetime"] == solo_date]
solo_fifteen_df = data_15min[data_15min["Datetime"] == solo_date]

print(solo_hourly_df)
print(solo_fifteen_df)

In [ ]:
# Create a single chart of the either of the data sets
def solo_day_chart(df, day):
    # Filter data for the specific day
    #     day = '2018-01-03'
    df_day = df[df["Datetime"].str.contains(day)]

    # Create the Altair chart
    chart = (
        alt.Chart(df_day)
        .mark_line()
        .encode(
            x="Datetime:T",
            y="value:Q"
            # y='Power at point in time MW'
            # title=day
        )
        .properties(width=600, height=300)
        .configure_axis(grid=True)
    )

    display(chart)


# create a stacked chart of both of the datasets


def min_hour_chart(df_min, df_hour, day):

    df_day_min = df_min[df_min["Datetime"].str.contains(day)]
    df_day_hour = df_hour[df_hour["Datetime"].str.contains(day)]

    # Merge the DataFrames on the 'Datetime' column
    merged_df = pd.merge(
        df_day_min, df_day_hour, on="Datetime", how="outer", suffixes=("_min", "_hour")
    )

    # Select the required columns
    result_df = merged_df[["Datetime", "value", "Power at point in time MW"]]

    # Convert 'Datetime' column to datetime data type
    result_df["Datetime"] = pd.to_datetime(result_df["Datetime"])

    # Sort the DataFrame by 'Datetime' column
    # result_df = result_df.sort_values(by='Datetime')

    chart1 = (
        alt.Chart(df_day_hour)
        .mark_line()
        .encode(
            x="Datetime:T",
            y=alt.Y("value:Q", scale=alt.Scale(domain=(0, 80000)))
            # y='Power at point in time MW'
            # title=day
        )
        .properties(width=1200, height=600, title=alt.TitleParams(text=day))
    )

    chart2 = (
        alt.Chart(df_day_min)
        .mark_line()
        .encode(
            x="Datetime:T",
            y=alt.Y("Power at point in time MW:Q", scale=alt.Scale(domain=(0, 300))),
            color=alt.value("red")
            # title=day
        )
        .properties(width=1200, height=600, title=alt.TitleParams(text=day))
    )

    combined_chart = (chart1 + chart2).resolve_scale(y="independent")
    display(combined_chart)

    return result_df


# result_df = min_hour_chart(data_15min, data_hourly, "2018-12-05")

In [ ]:
# solo_day_chart(data_hourly,"2018-12-05")
# result_df = min_hour_chart(data_15min, data_hourly, "2018-12-11")

result_df = min_hour_chart(fmintohour_MW, data_hourly, "2018-12-05")

# combined_chart.save('my_chart.html')

# import webbrowser
# webbrowser.open('my_chart.html')

In [ ]:
print(result_df)
result_df.to_csv("data/island_A/exported/solo_day_v1.csv", index=False)

In [ ]:
# ------ plot a range of graphs on a grid --------


def min_hour_chart_grid(df_min, df_hour, day):
    df_day_min = df_min[df_min["Datetime"].str.contains(day)]
    df_day_hour = df_hour[df_hour["Datetime"].str.contains(day)]

    chart1 = (
        alt.Chart(df_day_hour)
        .mark_line(color="blue")
        .encode(x="Datetime:T", y=alt.Y("value:Q", scale=alt.Scale(domain=(0, 80000))))
        .properties(width=600, height=300, title=alt.TitleParams(text=day))
    )

    chart2 = (
        alt.Chart(df_day_min)
        .mark_line(color="red")
        .encode(
            x="Datetime:T", y=alt.Y("Power at point in time MW:Q", scale=alt.Scale(domain=(0, 300)))
        )
    )

    # Custom legend
    legend = (
        alt.Chart(
            pd.DataFrame(
                [
                    {"label": "Hourly Data", "color": "blue"},
                    {"label": "15-min Data", "color": "red"},
                ]
            )
        )
        .mark_point()
        .encode(
            y=alt.Y("label:N", axis=alt.Axis(orient="right")),
            color=alt.Color("color:N", scale=None, legend=None),
        )
    )

    combined_chart = alt.layer(chart1, chart2).resolve_scale(y="independent")

    return combined_chart


# New function to generate a grid of charts for a range of dates
def grid_charts(df_min, df_hour, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")
    charts = []

    for date in date_range:
        day = date.strftime("%Y-%m-%d")
        chart = min_hour_chart_grid(df_min, df_hour, day)
        charts.append(chart)

    # Arrange the charts in a grid
    rows = []
    row_size = 8
    for i in range(0, len(charts), row_size):
        row = alt.hconcat(*charts[i : i + row_size])
        rows.append(row)

    grid = alt.vconcat(*rows)

    display(grid)


# grid_charts(data_15min, data_hourly, '2018-11-01', '2019-01-02')

In [ ]:
grid_charts(data_15min, data_hourly, "2018-11-01", "2019-01-02")

In [ ]:
grid_charts(fmintohour_MW, data_hourly, "2018-11-01", "2018-11-04")

In [ ]:
grid_charts(fmintohour_MW, data_hourly, "2018-11-01", "2019-01-02")